# Example 3: Golden crossover strategy

## Pre-requisites
### 1. If you have not opened the notebook in Colab, select the button below
<a href="https://githubtocolab.com/SIGTechnologies/sigtech-python/blob/master/examples/3_Golden_Crossover.ipynb">
    <img src="https://sigtech.com/wp-content/uploads/2023/08/grey_google_colab.svg"></a>

### 2. Enter your API key
After pasting in your API key, you need to run the cell. In Colab, hover your cursor over an individual code cell and click play to run it.
>**Tip**!\
>After pasting in your API key, you can press `CTRL-F9` (Windows) or `⌘-F9` (Mac) to run the entire notebook at once.

In [ ]:
# Install our Python SDK
!pip install sigtech 

# Import OS and our Python SDK
import sigtech.api as sig
import os

# Define your API key as a string. Remember to delete it before sharing your notebook with others. Replace 
# <YOUR_API_KEY> with the API key you have generated. e.g. os.environ['SIGTECH_API_KEY'] = 'sig_A1B2C3D4E5f6g7h8i9'
os.environ['SIGTECH_API_KEY'] = '<YOUR_API_KEY>'

### 3. Set up your Colab environment

In [ ]:
# Import any additional Python libraries you require.
import datetime as dtm
import pandas as pd
import numpy as np

### 4. Create a session
After installing our Python SDK, defining your API key, importing any additional Python libraries or functions you require, and setting any default parameters, initialize your session.

In [ ]:
sig.init()

## Introduction to the golden crossover strategy
This is a popular technical analysis based trading strategy. It involves the use of moving averages to identify buy signals.

The three main components of the Golden Crossover strategy are:
- **Simple moving averages (SMAs)**: moving averages are calculated by taking the average price of an asset over a specified period. In the case of the golden crossover strategy, two specific SMAs are used: the short-term moving average and the long-term moving average.
- **Golden crossover**: the golden crossover occurs when the short-term moving average crosses above the long-term moving average. It is considered a bullish signal.
- **Death cross**: occurs when the short-term moving average crosses below the long-term moving average. It is considered a bearish signal.


## Our strategy
- Will generate and test a golden cross strategy for trading the E-mini S&P 500 (ES) futures contract. 
- The strategy will use two simple moving averages (SMAs) with different time periods (60 days and 1 year) to generate buy and sell signals based on golden crossover and death cross events.

## 1. Create the simple moving averages   
First, we define the futures contract. This done using the `RollingFutureStrategy` class. 

>The `contract_code` and `contract_sector` can be found in our [API data catalog](https://sigtechapi.streamlit.app/).

In [ ]:
es = sig.RollingFutureStrategy(
        contract_code='ES',
        contract_sector='INDEX',
        currency='USD',
        start_date='2015-01-01'
    )
es.history() # Function from our library which calls the historical values of the futures contract.

Next we need to define the short-term and long-term moving average.

The following code calculates the 10-day percentage change of the ES futures price and adds it as a new column to `total_df`. Two new columns are added to `total_df`, representing the 60-day and 1-year simple moving averages (SMA) of the ES futures price. Rows with any missing data are dropped.

In [ ]:

total_df = pd.DataFrame(es.history().rename(es.name)).dropna()

total_df['10d_pct_chg'] = total_df[es.name].pct_change(10)*100
total_df['60d_sma'] = total_df[es.name].rolling(60).mean().dropna()
total_df['1yr_sma'] = total_df[es.name].rolling(252).mean().dropna()
total_df = total_df.dropna()
total_df.tail(10)

We can generate a plot for the ES futures price, the 60-day SMA, and the 1-year SMA using `.plot()`.

In [ ]:
total_df.plot(y=[es.name,'60d_sma','1yr_sma'],figsize=(20,10))

## 2. Generate a signal for the golden crossover
First we need to convert our DataFrame into one which is compatible with `SignalStrategy`. 

> A `SignalStrategy` requires a signal DataFrame as an input (this is a pandas DataFrame where the column headers are the instrument names and the values are the signals for each of the instruments). These signals can be either a number of units *or* a weight.

The following codeblock generates a new column in `total_df` called `sma_signal`, which assigns a value of 1 when the 60-day SMA is greater than the 1-year SMA, and -1 otherwise. This represents the golden crossover (1) and death cross (-1) signals, respectively.

`.head` shows what a signal DataFrame compatible with `SignalStrategy` looks like.

In [ ]:
total_df['sma_signal'] = np.where((total_df['60d_sma'] > total_df['1yr_sma']),1,-1)
total_df.head(10)

## 3. Backtest the strategy
Then we create our `SignalStrategy` using `total_df` as the `signal_input`. The following strategy rebalances every month. 

In [ ]:
signal = sig.SignalStrategy(
    currency='USD',
    signal_input=total_df[['sma_signal']].rename(columns={'sma_signal':es.name}),
    start_date=total_df.first_valid_index().date(),
    rebalance_frequency='EOM',
)

We can view the historical performance of our strategy using `.history()` and plot these values using `.plot()`.

In [ ]:
signal.history().plot(figsize=(20,10))